In [1]:
import argparse
import json
import os
import random
from gensim.summarization import bm25
import jieba
import numpy as np
from summary import stopwords, split_sentence, unique_sentences
from icecream import ic
import os
import os.path as osp
import numpy as np
import json
import torch
import re
from tqdm import tqdm
from icecream import ic
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../models/chinese-roberta-wwm-ext")
crime_pattern = re.compile(r'已构成(.*?)罪')

In [2]:
example_query = "2018年1月15日14时10分许，被告人莫新国酒后驾驶湘A×××××号小型轿车沿长沙市天心区伊莱克斯大道由南往北行驶至水电八局基地路段时被在该处执勤的长沙市公安局交通警察支队民警检查，经现场酒精吹气检测，测试结果显示其血液中乙醇含量为195毫克／100毫升，随即被告人莫新国被交警带至湖南省融城医院抽取血样，并将血样送至长沙市公安局物证鉴定所检验，经检验，其血液中乙醇含量为201.1毫克／100毫升。2009年11月15日，被告人莫新国经长沙市残疾人联合会审核为精神残疾人。2018年5月28日，经湖南省芙蓉司法鉴定中心鉴定，被告人莫新国作案时处于普通醉酒状态，实施危害行为时有完全刑事责任能力。2018年1月30日，被告人莫新国主动到公安机关投案，其归案后如实供述了自己的罪行。"

example_candidate = " 福州市鼓楼区人民检察院指控，1、2017年3月7日21时30分左右，被告人林隆川在福州市晋安区鹤林新城小区对面公交车站以人民币300元的价格将一袋净重0.64克的甲基苯丙胺（冰毒）卖给胡某4。 2、同日22时左右，胡某，4再次在上述地点以人民币300元的价格向被告人林隆川购买了一袋净重0.62克的甲基苯丙胺。 3、同日23时30分左右，陈某，4与被告人林隆川联系以人民币300元的价格购买一袋净重0.68克的甲基苯丙胺，被告人林隆川在福州市晋安区鹤林新城某座楼下交付毒品时，被公安民警当场抓获。 被告人林隆川三次贩卖毒品净重共1.94克。经鉴定：上述缴获的毒品均含有甲基苯丙胺成份。 对上述指控，公诉机关当庭宣读和出示了相关证据。通过举证，公诉机关认为被告人林隆川的行为已触犯了《中华人民共和国刑法》第三百四十七条第一款、第四款之规定，应以贩卖毒品罪追究其刑事责任，因被告人林隆川系累犯，应当从重处罚，建议对其在有期徒刑三年六个月至四年六个月的幅度内处刑，并处罚金，提请法院依法予以惩处。 被告人林隆川在庭审过程中对起诉书指控的第1、2起犯罪事实没有异议，但认为起诉书指控的第3起不是事实，辩称之前其曾答应陈某，4贩卖一个毒品给他，但是后来改变主意不想卖给陈某，4了，其当时下楼是为了买宵夜。 经审理查明，1、2017年3月7日21时30分左右，被告人林隆川在福州市晋安区鹤林新城小区对面公交车站以人民币300元的价格将一袋净重0.64克的甲基苯丙胺（冰毒）卖给胡某，4。 2、同日22时左右，胡某，4再次在上述地点以人民币300元的价格向被告人林隆川购买了一袋净重0.62克的甲基苯丙胺。 3、同日23时30分左右，陈某，4与被告人林隆川联系以人民币300元的价格购买一袋净重0.68克的甲基苯丙胺，被告人林隆川在福州市晋安区鹤林新城某座楼下交付毒品时，被公安民警当场抓获。 上述事实有下列经庭审举证、质证的证据证实： 1、证人陈某，4的证言及相关辨认笔录证实：2017年3月7日下午14时许，其通过微信联系胡某，4问他是否有冰毒可以拿，胡某，4答复可以带其去找一个朋友以一个冰毒人民币300元的价格买，其便到派出所举报了这件事。在民警的安排下，当天下午18时许，其在福州市鼓楼区洪山桥头附近中石化加油站接到胡某，4以及胡某，4带来的小弟杨某，4，当时其负责开车，胡某，4坐在副驾驶位，杨某，4坐在后排，三人一起到福州市晋安区鹤林新城对面的公交站等贩卖毒品的人出现。途中胡某，4表示要先将人民币300元通过微信转给贩卖毒品的那人，其便给胡某，4现金人民币300元，胡某，4通过微信转了人民币300元给卖毒品的人。2017年3月7日晚上21时许，其在鹤林新城对面公交站看见贩卖毒品的人驾驶着一部黑色现代牌小车出现，那人将车开到其车旁，放下玻璃，扔出一个蓝色七匹狼烟盒便开车走了。其下车捡起烟盒并交给胡某，4，胡某，4打开烟盒从里面拿出一个用透明自封袋包装的白色晶体交给其，之后其开车带着胡某，4、杨某，4离开鹤林新城，途中，其给胡某，4、杨某，4各200元人民币辛苦费，杨某，4没有收，胡某，4收下后说一个冰毒太少不够玩，主动联系刚才贩卖毒品的人通过微信转账人民币300元再买一个冰毒，还让其把车开回鹤林新城对面的公交站。2017年3月7日晚上22时左右，其开车带着胡某，4、杨某，4回到鹤林新城对面的公交站，发现刚才贩卖毒品的人已经开着车停在那里等，其就将车靠过去，那人和上次一样把玻璃放下扔出一个纸团后将车开走。那人在放下玻璃时其突然认出是被告人林隆川，微信名称是“政府高级官员”。胡某，4将纸团内的一袋透明自封袋包装白色晶体放进自己上衣口袋内后，其开车带着胡某，4、杨某，4离开，随后民警将胡某，4、杨某，4抓获。后来民警得知其认识刚才贩卖毒品的被告人林隆川，便让其将被告人林隆川引出来，其立即与被告人林隆川微信联系并转账给他人民币300元购买冰毒，被告人林隆川让其到鹤林新城某栋楼下等他，其和民警到指定地点后没多久被告人林隆川就下楼了，民警迅速将被告人林隆川抓获并从他身上查获一袋透明自封袋包装的白色晶体。 2、证人胡某，4的证言及相关辨认笔录证实：2017年3月7日下午16时左右，陈某，4微信询问其是否有地方可以买冰毒，其让陈某，4和其一起找被告人林隆川拿冰毒，其和被告人林隆川商定以人民币300元的价格购买一个冰毒，并约定好在福州市晋安区鹤林新城对面的公交站交易。当晚18时许，陈某，4开车在洪山桥头附近接上其和其朋友杨某，4，三人一起坐车到福州市晋安区鹤林新城对面公交站旁边等被告人林隆川，途中陈某，4将现金人民币300元给其，其通过微信转账人民币300元给被告人林隆川。当晚21时许，被告人林隆川开车来到其车边上，将窗户放下扔出一个蓝色七匹狼烟盒，然后就开车走了。陈某，4下车捡起烟盒交给其，其从烟盒拿出一袋塑料自封袋包装的白色晶体，确认是冰毒后交给陈某4，陈某，4拿到冰毒后开车带着其和杨某，4离开公交站，在车上陈某，4觉得一个冰毒不够玩让其也买一个，其找杨某，4凑足人民币300元通过微信转账给被告人林隆川，同时和被告人林隆川约定交易地点还是在福州市晋安区鹤林新城对面的公交站。2017年3月7日22时左右，陈某，4驾车载着其和杨某，4来到约定地点，被告人林隆川现代牌的车已经在公交站等待，被告人林隆川见其车靠近，将车窗放下扔出一团纸巾后就离开了。杨某，4下车将纸团捡起交给其，其打开纸巾发现里面是一袋塑料自封袋包装的白色晶体便放进自己上衣口袋，陈某，4开车带其离开，之后警察将其抓获。 3、证人杨某，4的证言及辨认笔录证实，2017年3月7日18时许，胡某，4告诉其他的朋友要买冰毒，让其一起去。之后，胡某，4的朋友开车载着其和胡某，4到福州市晋安区鹤林新城公交站旁和贩卖毒品的人交易。在车上胡某，4一直和贩卖毒品的人联系，并转账人民币300元给卖毒人，当晚21时许，一辆黑色小车停在其车旁边，开车男子打开车窗后扔出一个蓝色烟盒后开车离开。胡某，4的朋友捡起烟盒拿给胡某，4，胡某，4拿出烟盒里的白色塑料自封袋又交给他朋友，三人就驾车离开。途中，开车的男子对胡某，4说一个冰毒怕不够玩，能否再买一个。胡某，4又联系卖毒人转账人民币300元再买个毒品，交易地点还是约定在鹤林新城公交站。当其车到达交易地点时，那个卖毒品的人已经开车在那里等待，他朝其车子挡风玻璃扔了一包冰毒后开车离开。其将冰毒捡起来交给胡某，4，胡某，4朋友开车带着其和胡某，4离开，之后其、胡某，4就被警察抓获。 4、被告人林隆川在公安机关的原供述称：2017年3月7日晚上7点左右，胡某，4联系其购买冰毒并通过微信转账给其人民币300元，双方约定在鹤林新城对面的公交车站交易，其到福州市晋安区麻将馆内找到“黄某”，以人民币1200元的价格购买了四个冰毒。拿完冰毒其回到福州市晋安区鹤林新城某栋的家中，胡某，4微信催其交易，其驾驶闽A×××××号现代小车到鹤林新城对面公交车站，看到胡某，4的车停在那里，其将一袋冰毒扔向胡某，4的车子后离开。大约是当晚10点，其回到家中后胡某，4又联系其再买一袋冰毒，并通过微信转账给其人民币300元，约定交易地点不变。其再次驾驶闽A×××××号现代小车到了之前交易的公交站附近，几分钟后胡某，4的车停到其车边上，其将一袋冰毒往对方小车挡风玻璃上一扔便开车离开。其回到家中玩了一个多小时，陈某，4转账人民币300元给其向其购买冰毒，其让他到鹤林新城某号楼楼下拿冰毒，其拿了一袋冰毒到一楼准备卖给陈某，4时被公安民警抓获。 5、扣押的涉案毒品照片等物证、福州市公安局鼓楼分局出具的扣押清单、福建省毒品实物缴交收据等书证及福建正中司法鉴定所出具的毒品定性司法鉴定检验报告书，证实被告人林隆川贩卖的冰毒中检出甲基苯丙胺成分，数量为1.94克，现毒品已上缴。 6、微信聊天记录、微信支付记录等电子数据及相关辨认笔录，证实被告人林隆川与证人胡某，4、陈某，4协商交易冰毒、收取毒资的情况。 7、到案经过，证实被告人林隆川系由公安机关抓获归案。 8、户籍证明、前科材料等其他证据材料。 上述证据本院予以确认。 "

In [3]:
def cut_words(sentence):
    words = jieba.cut(sentence, cut_all=False)
    tem = " ".join(words).split()
    return [i for i in tem if not i in stopwords]


def rerank_sentences(query_words, candidate):
    candidate_sentences = list(split_sentence(candidate))
    corpus = [cut_words(s) for s in candidate_sentences]
    bm25Model = bm25.BM25(corpus)
    sentence_scores = np.array(bm25Model.get_scores(query_words))
    filtered_candidate_sentences = [sent for sent, score in zip(
        candidate_sentences, sentence_scores) if score > 0]
    filtered_sentence_scores = sentence_scores[sentence_scores > 0]
    reranked_sentences = [filtered_candidate_sentences[i]
                          for i in filtered_sentence_scores.argsort().tolist()[::-1]]
    uniqued_reranked_sentences = list(unique_sentences(reranked_sentences))
    return ''.join(uniqued_reranked_sentences)


rerank_sentences(cut_words(example_query), example_candidate)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.624 seconds.
Prefix dict has been built successfully.


'2017年3月7日晚上22时左右，其开车带着胡某，4、其再次驾驶闽A×××××号现代小车到了之前交易的公交站附近，几分钟后胡某，4的车停到其车边上，其将一袋冰毒往对方小车挡风玻璃上一扔便开车离开。拿完冰毒其回到福州市晋安区鹤林新城某栋的家中，胡某，4微信催其交易，其驾驶闽A×××××号现代小车到鹤林新城对面公交车站，看到胡某，4的车停在那里，其将一袋冰毒扔向胡某，4的车子后离开。2017年3月7日21时30分左右，被告人林隆川在福州市晋安区鹤林新城小区对面公交车站以人民币300元的价格将一袋净重0.64克的甲基苯丙胺（冰毒）卖给胡某，4。2017年3月7日22时左右，陈某，4驾车载着其和杨某，4来到约定地点，被告人林隆川现代牌的车已经在公交站等待，被告人林隆川见其车靠近，将车窗放下扔出一团纸巾后就离开了。证人杨某，4的证言及辨认笔录证实，2017年3月7日18时许，胡某，4告诉其他的朋友要买冰毒，让其一起去。2017年3月7日下午16时左右，陈某，4微信询问其是否有地方可以买冰毒，其让陈某，4和其一起找被告人林隆川拿冰毒，其和被告人林隆川商定以人民币300元的价格购买一个冰毒，并约定好在福州市晋安区鹤林新城对面的公交站交易。2017年3月7日下午14时许，其通过微信联系胡某，4问他是否有冰毒可以拿，胡某，4答复可以带其去找一个朋友以一个冰毒人民币300元的价格买，其便到派出所举报了这件事。福建省毒品实物缴交收据等书证及福建正中司法鉴定所出具的毒品定性司法鉴定检验报告书，证实被告人林隆川贩卖的冰毒中检出甲基苯丙胺成分，数量为1.94克，现毒品已上缴。被告人林隆川在公安机关的原供述称：2017年3月7日晚上21时许，其在鹤林新城对面公交站看见贩卖毒品的人驾驶着一部黑色现代牌小车出现，那人将车开到其车旁，放下玻璃，扔出一个蓝色七匹狼烟盒便开车走了。被告人林隆川在庭审过程中对起诉书指控的第1、2017年3月7日晚上7点左右，胡某，4联系其购买冰毒并通过微信转账给其人民币300元，双方约定在鹤林新城对面的公交车站交易，其到福州市晋安区麻将馆内找到“黄某”，以人民币1200元的价格购买了四个冰毒。经鉴定：同日23时30分左右，陈某，4与被告人林隆川联系以人民币300元的价格购买一袋净重0.68克的甲基苯丙胺，被告人林隆川在福州市晋安区鹤林新城某座楼下交付毒品时，被公安民警当场

In [4]:
def preprocess_data(data_path, has_label=False):
    query_path = osp.join(data_path, 'query.json')
    all_candidates_path = osp.join(data_path, 'candidates')
    if has_label:
        label_path = osp.join(data_path, 'label_top30_dict.json')
        label = json.load(open(label_path))
        labels = []
    edges, inputs, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs = [
    ], [], [], [], [], []
    with open(query_path) as f:
        query_lines = f.readlines()
    for query_line in tqdm(query_lines):
        query_line = query_line.strip()
        query_dict = json.loads(query_line)
        input_str = "。".join(query_dict['crime']) + \
            "。"+query_dict['q']
        query_words = cut_words(query_dict['q'])
        tokenized_inputs = tokenizer(input_str, return_tensors="pt")
        query_idx = len(inputs)
        inputs.append(tokenized_inputs)
        node_graph_ids.append(len(query_ridxs))
        query_ridxs.append(query_dict['ridx'])
        query_ridx = str(query_dict['ridx'])
        candidates_path = osp.join(all_candidates_path, query_ridx)
        for candidate in os.listdir(candidates_path):
            candidate_ridx = candidate[:-5]
            candidate_path = osp.join(candidates_path, candidate)
            candidate_dict = json.load(open(candidate_path))
            all_text = ''.join(candidate_dict.values())
            crime_name = crime_pattern.search(all_text)
            if crime_name is None:
                crime_name = ''
            else:
                crime_name = crime_name.group(1) + '罪'
            candidate_text = rerank_sentences(
                query_words, candidate_dict['ajjbqk'])
            candidate_text = '。'.join(
                [crime_name, candidate_text])
            tokenized_candidate = tokenizer(
                candidate_text, return_tensors="pt")
            candidate_idx = len(inputs)
            inputs.append(tokenized_candidate)
            node_graph_ids.append(node_graph_ids[-1])
            edge_graph_ids.append(node_graph_ids[-1])
            edges.append([query_idx, candidate_idx])
            candidate_ridxs.append(int(candidate_ridx))
            if has_label:
                if candidate_ridx in label[query_ridx]:
                    labels.append(label[query_ridx][candidate_ridx])
                else:
                    labels.append(0)
    if has_label:
        return inputs, edges, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs, labels
    return inputs, edges, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs


train_path = '../data/origin/train'
train_inputs, train_edges, train_query_ridxs, train_node_graph_ids, train_edge_graph_ids, train_candidate_ridxs, train_labels = preprocess_data(
    train_path, has_label=True)


100%|██████████| 197/197 [07:11<00:00,  2.19s/it]


In [5]:
def save(processed_path, edges, inputs, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs, labels=None):
    torch.save(inputs, osp.join(processed_path, 'inputs.pt'))
    torch.save(edges, osp.join(processed_path, 'edges.pt'))
    torch.save(query_ridxs, osp.join(processed_path, 'query_ridxs.pt'))
    torch.save(candidate_ridxs, osp.join(processed_path, 'candidate_ridxs.pt'))
    torch.save(node_graph_ids, osp.join(processed_path, 'node_graph_ids.pt'))
    torch.save(edge_graph_ids, osp.join(processed_path, 'edge_graph_ids.pt'))
    if labels is not None:
        torch.save(labels, osp.join(processed_path, 'labels.pt'))

In [6]:
train_processed_path = '../data/bm25/train/processed'
os.makedirs(train_processed_path, exist_ok=True)
save(train_processed_path, train_edges, train_inputs, train_query_ridxs,
     train_node_graph_ids, train_edge_graph_ids, train_candidate_ridxs, train_labels)

In [7]:
test_path = '../data/origin/test'
test_inputs, test_edges, test_query_ridxs, test_node_graph_ids, test_edge_graph_ids, test_candidate_ridxs = preprocess_data(
    test_path, has_label=False)
test_processed_path = '../data/bm25/test/processed'
os.makedirs(test_processed_path, exist_ok=True)
save(test_processed_path, test_edges, test_inputs, test_query_ridxs,
     test_node_graph_ids, test_edge_graph_ids, test_candidate_ridxs)

100%|██████████| 40/40 [00:30<00:00,  1.33it/s]


In [8]:
val_query_ridxs = train_query_ridxs[:17]
edge_split_idx = (np.array(train_edge_graph_ids) < 17).sum()
node_split_idx = (np.array(train_node_graph_ids) < 17).sum()
val_inputs = train_inputs[:node_split_idx]
val_edges = train_edges[:edge_split_idx]
val_labels = train_labels[:edge_split_idx]
val_candidate_ridxs = train_candidate_ridxs[:edge_split_idx]
val_node_graph_ids = train_node_graph_ids[:node_split_idx]
val_edge_graph_ids = train_edge_graph_ids[:edge_split_idx]
val_processed_path = '../data/bm25/val/processed'
os.makedirs(val_processed_path, exist_ok=True)
save(val_processed_path, val_edges, val_inputs, val_query_ridxs,
     val_node_graph_ids, val_edge_graph_ids, val_candidate_ridxs, val_labels)

In [ ]:
train_dev_query_ridxs = train_query_ridxs[17:]
train_dev_inputs = train_inputs[node_split_idx:]
train_dev_edges = np.array(train_edges[edge_split_idx:]) - node_split_idx
train_dev_edges = train_dev_edges.tolist()
train_dev_labels = train_labels[edge_split_idx:]
train_dev_candidate_ridxs = train_candidate_ridxs[edge_split_idx:]
train_dev_node_graph_ids = np.array(train_node_graph_ids[node_split_idx:]) - 17
train_dev_edge_graph_ids = np.array(train_edge_graph_ids[edge_split_idx:]) - 17
train_dev_node_graph_idx = train_dev_node_graph_ids.tolist()
train_dev_edge_graph_idx = train_dev_edge_graph_ids.tolist()
train_dev_processed_path = '../data/bm25/train_dev/processed'
os.makedirs(train_dev_processed_path, exist_ok=True)
save(train_dev_processed_path, train_dev_edges, train_dev_inputs,
     train_dev_query_ridxs, train_dev_node_graph_ids, train_dev_edge_graph_ids,
     train_dev_candidate_ridxs, train_dev_labels)